 <center> <h1> <b> Pattern Recognition and Machine Learning (EE5607) </b> </h1> </center>

<b> Programming Assignment - 04 - Neural Networks</b>

<b> Instructions </b>
1. Plagiarism is strictly prohibited.
2. Delayed submissions will be penalized with a scaling factor of 0.5 per day.
3. Please DO NOT use any machine learning libraries unless and otherwise specified.



<b> Assignment on Neural Networks </b> <br>
This programming assignment gives you a chance to perform the classification task using neural networks. You will get to build a neural network from scratch and train and test it on a standard classification dataset. Further you will learn different tricks and techniques to train a neural network eficiently by observing few important issues and trying to overcome them. This includes observing the performance of the network for different activation functions and optimization algorithms. We will conclude with implementation of various regularization techniques and ResNet to overcome the problems of overfitting and vanishing gradients.

In [ ]:
#All imports
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import collections

<b>  Part - (1) : Train a classification network from scratch </b>
1. Load MNIST data and create train, test splits
2. Design a simple classification network


> Network should consists of three Dense Layers with 512 nodes. The same architecture is used throughout the assignment to understand the effect of hyper parameters.

> Use stochastic gradient descent optimization algorithm to update the parameters. You can use the learning rate suitable for the MNIST digit classification problem.

> Use ReLU activation function in the first two layers. Softmax activation function should be used at the last layer to get the posterior probability of the classes.






3. Train the network using MNIST training data and evaluate the performance on MNIST test data



In [ ]:
##################################################
#Load MNIST data.
##################################################
#Mount the drive
from google.colab import drive
from mlxtend.data import loadlocal_mnist
import matplotlib.pyplot as plt

drive.mount('/content/drive',force_remount=True)
root_path='/content/drive/MyDrive/MNIST'
train_data_path = str(root_path) + "/" + 'train-images.idx3-ubyte'
train_label_path = str(root_path) + "/" + 'train-labels.idx1-ubyte'
test_data_path = str(root_path) + "/" + 't10k-images.idx3-ubyte'
test_label_path = str(root_path) + "/" + 't10k-labels.idx1-ubyte'

X_train, Y_train = loadlocal_mnist(images_path=train_data_path, labels_path=train_label_path)
X_test, Y_test = loadlocal_mnist(images_path=test_data_path, labels_path=test_label_path)

no_I=10000
x_train = X_train[:no_I,:]
y_train = Y_train[:no_I]
x_test = X_test[:2000,:]
y_test = Y_test[:2000]
y_train=y_train.reshape((len(y_train),1))
y_test=y_test.reshape((len(y_test),1))

##################################################
#Build the architecture
##################################################

import numpy as np
import random

def ReLU(m,derivative = False):
  if derivative:
    return np.where(m <= 0, 0, 1)
  else:
   return np.maximum(m,0)

def softmax(x, derivative = False):
  max = np.max(x,axis=1,keepdims=True) #returns max of each row and keeps same dims
  e_x = np.exp(x - max) #subtracts each row with its max value
  sum = np.sum(e_x,axis=1,keepdims=True) #returns sum of each row and keeps same dims
  f_x = e_x / sum
  if derivative:
    return f_x*(1-f_x)
  else:
    return f_x


def forward_and_back(x_ip,w1,w2,w3,y_tr,neta,train=True):
  #######forward
  ah1=w1@x_ip.T
  z1=ReLU(ah1,derivative=False)
  ah2=w2@z1
  z2=ReLU(ah2,derivative=False)
  ah3=w3@z2
  op_pr=softmax(ah3,derivative=False)
  # print("w shapes",np.shape(w1),np.shape(w2),np.shape(w3))
  # print("Z shapes",np.shape(x_ip),np.shape(z1),np.shape(z2),np.shape(op_pr))
  # print("ah shapes",np.shape(ah1),np.shape(ah2),np.shape(ah3))
  y_pr = np.argmax(op_pr,axis=0)
  y_pr=y_pr.reshape(y_tr.shape)
  ######backprop
  if (train):
    dj_da3=softmax(ah3,derivative=True)*((y_pr-y_tr).T)
    dj_dw3=(dj_da3)@z2.T
    w3=w3-neta*dj_dw3
    dj_da2=ReLU(ah2,derivative=True)*(w3.T@dj_da3)
    dj_dw2=dj_da2@z1.T
    w2=w2-neta*dj_dw2
    dj_da1=ReLU(ah1,derivative=True)*(w2.T@dj_da2)
    dj_dw1=dj_da1@x_ip
    w1=w1-neta*dj_dw1
    return w1,w2,w3
  else:
    print("for testing")
    return y_pr

##################################################
#Train the network
##################################################
def train_mlp(x_train,y_train,epochs,h1_nodes,h2_nodes,op_nodes, batch_size):
  neta=0.001
  X_Y=np.hstack((x_train,y_train))
  wt1=np.random.randint(0,10,(h1_nodes,784))
  wt2=np.random.randint(0,10,(h2_nodes,h1_nodes))
  wt3=np.random.randint(0,10,(op_nodes,h1_nodes))
  for i in range(epochs):
    np.random.shuffle(X_Y)
    for j in range(100):
      xtr_b=X_Y[int(j*batch_size):int((j+1)*batch_size),:]
      ytr_b=X_Y[int(j*batch_size):int((j+1)*batch_size),-1].reshape(len(xtr_b),1)
      up_w1,up_w2,up_w3=forward_and_back(xtr_b[:,:-1],wt1,wt2,wt3,ytr_b,neta,train=True)
      wt1=up_w1
      wt2=up_w2
      wt3=up_w3
  return wt1,wt2,wt3
wop1,wop2,wop3=train_mlp(x_train,y_train,10,512,512,10,100)

##################################################
#Test the network and write down the performance
##################################################
def test_mlp(wts1,wts2,wts3,x_tst,y_tst):
  neta=0.001
  y_=forward_and_back(x_tst,wts1,wts2,wts3,y_tst,neta,train=False)
  t_t=len(y_)*0.1
  e_=np.sum(y_!=y_tst)
  ac=(len(y_tst)-e_)/t_t
  print("Acurracy in % : ",ac*100)

test_mlp(wop1,wop2,wop3,x_test,y_test)


#######observations
##the accuracy here depends on many factors like weights initialization, activation function, dataset size that is considered,batch size, epochs etc.
#here the accuracy is found to be approximatly 85-88 percent










Mounted at /content/drive
for testing
Acurracy in % :  87.5


<b> Part - (2) : Understanding activation functions </b>

In this part you will learn to use different activation functions for the classification task and compare their performances.

1. Train MNIST digit classification problem with different activation functions i.e. Sigmoid, Tanh, ReLU, LeakyReLU etc. You can stick to stochastic gradient descent optimization algorithm for this part
2. Report the accuray on MNIST test data for all the experiments. Write down your observations in the report.</br>
NOTE: You can use the classification network designed by you from scratch (or) you can build the model using pytorch.



In [ ]:
#########################################
##  loading data using torch
#########################################
import torch
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transform))
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transform))

##################################################
#Train the network with different activation functions
##################################################
import time
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


class Net(nn.Module):
    def __init__(self, epochs=10):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, 512)
        self.linear3 = nn.Linear(512, 10)

        self.epochs = epochs

    def forward_pass_relu(self, x):   #########ReLu Activation function
        x1 = self.linear1(x)
        x1 = torch.relu(x1)
        x1 = self.linear2(x1)
        x1 = torch.relu(x1)
        x1 = self.linear3(x1)
        x1 = torch.softmax(x1, dim=0)
        return x1
    def forward_pass_sig(self, x):      #########Sigmoid Activation function
        x2 = self.linear1(x)
        x2 = torch.sigmoid(x2)
        x2 = self.linear2(x2)
        x2 = torch.sigmoid(x2)
        x2 = self.linear3(x2)
        x2 = torch.softmax(x2, dim=0)
        return x2
    def forward_pass_tanh(self, x):       #########tanh Activation function
        x3 = self.linear1(x)
        x3 = torch.tanh(x3)
        x3 = self.linear2(x3)
        x3 = torch.tanh(x3)
        x3 = self.linear3(x3)
        x3 = torch.softmax(x3, dim=0)
        return x3
    def forward_pass_leky_relu(self, x):      #########leky_ReLu Activation function
        x4 = self.linear1(x)
        x4 = F.leaky_relu(x4)
        x4 = self.linear2(x4)
        x4 = F.leaky_relu(x4)
        x4 = self.linear3(x4)
        x4 = torch.softmax(x4, dim=0)
        return x4

    def one_hot_encode(self, y):
        encoded = torch.zeros([10], dtype=torch.float64)
        encoded[y[0]] = 1.
        return encoded

    def train(self, train_loader, optimizer, criterion):
        start_time = time.time()
        loss = None

        for iteration in range(self.epochs):
            for x,y in train_loader:
                y = self.one_hot_encode(y)
                optimizer.zero_grad()
                #print(x.shape,y.shape)
                output1 = self.forward_pass_relu(torch.flatten(x))
                loss1 = criterion(output1, y)
                loss1.backward()
                optimizer.step()
                optimizer.zero_grad()
                output2 = self.forward_pass_sig(torch.flatten(x))
                loss2 = criterion(output2, y)
                loss2.backward()
                optimizer.step()
                optimizer.zero_grad()
                output3 = self.forward_pass_tanh(torch.flatten(x))
                loss3 = criterion(output3, y)
                loss3.backward()
                optimizer.step()
                optimizer.zero_grad()
                output4 = self.forward_pass_leky_relu(torch.flatten(x))
                loss4 = criterion(output4, y)
                loss4.backward()
                optimizer.step()
            print('Epoch: {0}, Time Spent: {1:.2f}s, Acc_ReLu: {2:.2f}%, Acc_Sigmoid: {3:.2f}%, Acc_tanh: {4:.2f}%, Acuu_L_ReLu: {5:.2f}%'.format(
                iteration+1, time.time() - start_time, (1-loss1)*100,(1-loss2)*100,(1-loss3)*100,(1-loss4)*100)

            )

model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()
model.train(train_loader, optimizer, criterion)
##################################################
#Write down the accuracies and report your observations
# ##################################################
# Epoch: 1, Time Spent: 510.83s, Acc_ReLu: 30.83%, Acc_Sigmoid: 26.73%, Acc_tanh: 31.20%, Acuu_L_ReLu: 30.89%
# Epoch: 2, Time Spent: 1009.31s, Acc_ReLu: 32.97%, Acc_Sigmoid: 26.19%, Acc_tanh: 33.23%, Acuu_L_ReLu: 33.02%
# Epoch: 3, Time Spent: 1511.74s, Acc_ReLu: 33.61%, Acc_Sigmoid: 26.10%, Acc_tanh: 33.87%, Acuu_L_ReLu: 33.64%
# Epoch: 4, Time Spent: 2052.42s, Acc_ReLu: 33.99%, Acc_Sigmoid: 26.24%, Acc_tanh: 34.24%, Acuu_L_ReLu: 34.01%
# Epoch: 5, Time Spent: 2708.40s, Acc_ReLu: 34.22%, Acc_Sigmoid: 26.81%, Acc_tanh: 34.40%, Acuu_L_ReLu: 34.23%
# Epoch: 6, Time Spent: 3334.35s, Acc_ReLu: 34.34%, Acc_Sigmoid: 27.75%, Acc_tanh: 34.46%, Acuu_L_ReLu: 34.35%
# Epoch: 7, Time Spent: 3941.77s, Acc_ReLu: 34.39%, Acc_Sigmoid: 28.61%, Acc_tanh: 34.48%, Acuu_L_ReLu: 34.40%
# Epoch: 8, Time Spent: 4550.82s, Acc_ReLu: 34.43%, Acc_Sigmoid: 29.30%, Acc_tanh: 34.48%, Acuu_L_ReLu: 34.43%
# Epoch: 9, Time Spent: 5167.87s, Acc_ReLu: 34.45%, Acc_Sigmoid: 29.87%, Acc_tanh: 34.48%, Acuu_L_ReLu: 34.45%
# Epoch: 10, Time Spent: 5686.22s, Acc_ReLu: 34.46%, Acc_Sigmoid: 30.37%, Acc_tanh: 34.48%, Acuu_L_ReLu: 34.46%

#from the above results, tanh activation funtion found to produce better results when compared with the accuracy of other activation functions



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Epoch: 1, Time Spent: 510.83s, Acc_ReLu: 30.83%, Acc_Sigmoid: 26.73%, Acc_tanh: 31.20%, Acuu_L_ReLu: 30.89%
Epoch: 2, Time Spent: 1009.31s, Acc_ReLu: 32.97%, Acc_Sigmoid: 26.19%, Acc_tanh: 33.23%, Acuu_L_ReLu: 33.02%
Epoch: 3, Time Spent: 1511.74s, Acc_ReLu: 33.61%, Acc_Sigmoid: 26.10%, Acc_tanh: 33.87%, Acuu_L_ReLu: 33.64%
Epoch: 4, Time Spent: 2052.42s, Acc_ReLu: 33.99%, Acc_Sigmoid: 26.24%, Acc_tanh: 34.24%, Acuu_L_ReLu: 34.01%
Epoch: 5, Time Spent: 2708.40s, Acc_ReLu: 34.22%, Acc_Sigmoid: 26.81%, Acc_tanh: 34.40%, Acuu_L_ReLu: 34.23%
Epoch: 6, Time Spent: 3334.35s, Acc_ReLu: 34.34%, Acc_Sigmoid: 27.75%, Acc_tanh: 34.46%, Acuu_L_ReLu: 34.35%
Epoch: 7, Time Spent: 3941.77s, Acc_ReLu: 34.39%, Acc_Sigmoid: 28.61%, Acc_tanh: 34.48%, Acuu_L_ReLu: 34.40%
Epoch: 8, Time Spent: 4550.82s, Acc_ReLu: 34.43%, Acc_Sigmoid: 29.30%, Acc_tanh: 34.48%, Acuu_L_ReLu: 34.43%
Epoch: 9, Time Spent: 5167.87s, Acc_ReLu: 34.45%, Acc_Sigm

<b> Part - (3) : Understanding optimization algorithms </b>

In this part you will learn to use different optimiztion algorithm apart from SGD.

1. Using the best activation function from Part - (2), train the classification network using Adam optimization algorithm.
3. Compare the accuracy of the networks trained with SGD and Adam optimization algorithms.
4. Report your observations.

In [ ]:
##################################################
#Train the network using Adam optimizer
##################################################
import time
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


class Net2(nn.Module):
    def __init__(self, epochs=5):
        super(Net2, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, 512)
        self.linear3 = nn.Linear(512, 10)
        self.epochs = epochs

    def forward_pass_tanh(self, x):       #########tanh Activation function
        x3 = self.linear1(x)
        x3 = torch.tanh(x3)
        x3 = self.linear2(x3)
        x3 = torch.tanh(x3)
        x3 = self.linear3(x3)
        x3 = torch.softmax(x3, dim=0)
        return x3

    def one_hot_encode(self, y):
        encoded = torch.zeros([10], dtype=torch.float64)
        encoded[y[0]] = 1.
        return encoded

    def train(self, train_loader, optimizer3, criterion):
        start_time = time.time()
        loss = None

        for iteration in range(self.epochs):
            for x,y in train_loader:
                y = self.one_hot_encode(y)
                optimizer3.zero_grad()
                output3 = self.forward_pass_tanh(torch.flatten(x))
                loss3 = criterion(output3, y)
                loss3.backward()
                optimizer3.step()

            #print('Epoch: {0}, Time Spent: {1:.2f}s, Acc_tanh: {2:.2f}%,'.format(iteration+1, time.time() - start_time, (1-loss3)*100))
    def test(self,test_loader,optimizer3,criterion):
        correct=0
        for xt,yt in test_loader:
              ytst = self.one_hot_encode(yt)
              optimizer3.zero_grad()
              output_tst = self.forward_pass_tanh(torch.flatten(xt))
              loss_tst = criterion(output_tst, ytst)
        print("accuracy :",(1-loss_tst)*100)


model2 = Net2()
optimizer1 = optim.SGD(model2.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()
optimizer2=optim.Adam(model2.parameters(), lr=0.001)

##################################################
#Test the network
##################################################
###training and testing using Adam optimizer
model2.train(train_loader, optimizer2, criterion)
print("using Adam optimizer accuracy is:\n")
model2.test(test_loader, optimizer2,criterion)

###training and testing using SGD optimizer
model2.train(train_loader, optimizer2, criterion)
print("using SGD optimizer accuracy is:\n")
model22.test(test_loader, optimizer1,criterion)

##################################################
#Compare the accuracies an deport your observations
##################################################

#################each code is written and run in seperate notebooks since the training is taking time and the observations and the results are written below
#accuracy using SGD is found to be 34.4727 with tanh activation function
##accuracy : tensor(34.4727, dtype=torch.float64, grad_fn=<MulBackward0>)


#accuracy using Adam optimizer is found to be 34.4841 with tanh activation function
#accuracy : tensor(34.4841, dtype=torch.float64, grad_fn=<MulBackward0>)
#Observation:As the accuracy of adam is higher than that of SGD optimizer,the  model performs slightly better with adam optimiser when compared to SGD optimizer.


using Adam optimizer accuracy is:

accuracy : tensor(34.4841, dtype=torch.float64, grad_fn=<MulBackward0>)


<b> Part - (4) : Understanding regularization methods </b>

In this part of the assignment, you will learn about a few regularization techniques to reduce the overfitting problem.</br>
Using the above built network, inculcate the following techniques to reduce the overfitting by retraining the network efficiently. Write down the accuracies for each case.

1. Weight regularization
2. Dropout with a probability of 0.5
3. Early stopping


In [ ]:
##################################################
#Training with weight regularization
##################################################

criterion = nn.BCEWithLogitsLoss()
optimizer_4=optim.Adam(model2.parameters(), lr=0.001,weight_decay=0.001)
model2.train(train_loader, optimizer_4, criterion)
print("using Adam optimizer with L2 regularization, the accuracy is:\n")
model2.test(test_loader, optimizer_4,criterion)

#################each code is witten and run in seperate notebooks since the training is taking time and the observations and the results are written below
########using Adam optimizer with L2 regularization, the accuracy is:34.4925
#accuracy : tensor(34.4925, dtype=torch.float64, grad_fn=<MulBackward0>)

In [ ]:

##################################################
#Training with dropout strategy
##################################################
##################################################
#Train the network using Adam optimizer
##################################################
###introducing dropout in the above code

class Net3(nn.Module):
    def __init__(self, epochs=5):
        super(Net3, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, 512)
        self.linear3 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)
        self.epochs = epochs

    def forward_pass_tanh(self, x):       #########tanh Activation function
        x3 = self.linear1(x)
        x3 = torch.tanh(x3)
        x3 = self.linear2(x3)
        x3 = self.dropout(x3)
        x3 = torch.tanh(x3)
        x3 = self.linear3(x3)
        x3 = self.dropout(x3)
        x3 = torch.softmax(x3, dim=0)
        return x3

    def one_hot_encode(self, y):
        encoded = torch.zeros([10], dtype=torch.float64)
        encoded[y[0]] = 1.
        return encoded

    def train(self, train_loader, optimizer3, criterion):
        start_time = time.time()
        loss = None

        for iteration in range(self.epochs):
            for x,y in train_loader:
                y = self.one_hot_encode(y)
                optimizer3.zero_grad()
                output3 = self.forward_pass_tanh(torch.flatten(x))
                loss3 = criterion(output3, y)
                loss3.backward()
                optimizer3.step()

            #print('Epoch: {0}, Time Spent: {1:.2f}s, Acc_tanh: {2:.2f}%,'.format(iteration+1, time.time() - start_time, (1-loss3)*100))
    def test(self,test_loader,optimizer3,criterion):
        correct=0
        for xt,yt in test_loader:
              ytst = self.one_hot_encode(yt)
              optimizer3.zero_grad()
              output_tst = self.forward_pass_tanh(torch.flatten(xt))
              loss_tst = criterion(output_tst, ytst)
        print("accuracy :",(1-loss_tst)*100)


model3 = Net3()


criterion = nn.BCEWithLogitsLoss()
optimizer2=optim.Adam(model3.parameters(), lr=0.001)
model3.train(train_loader, optimizer2, criterion)
print("using Adam optimizer with Dropout, the accuracy is:\n")
model3.test(test_loader, optimizer2,criterion)

#################each code is written run in seperate notebooks since the training is taking time and the observations and the results are written below
########using Adam optimizer with Dropout, the accuracy is:34.5305%
#accuracy : tensor(34.5305, dtype=torch.float64, grad_fn=<MulBackward0>)
 #model performs slightly better with dropout when compared to that with L2 regularization.


In [ ]:
##################################################
#Training with early stopping criterion
##################################################


<b> Part - (5) : Understanding vanishing gradients problem </b> </br>
Use the best trained models in part (4) to understand the problem of vanishing gradient.
1. Plot the norm of the gradients for various layers with/without weight regularization
3. Repeat the same experiments with skip connections. You can choose your desired skip connections in the network by increasing the number of hidden layers.
4. Report your observations on the differences between the plots in the above two cases.

In [ ]:
##################################################
#Plots without skip connections
##################################################















##################################################
#Plots with skip connections
##################################################












##################################################
#Observations
##################################################






